## Greenwich Area of Study

Select area in Greenwich municipal boundary

First: fix dependencies etc in OSM2AT

To include

- OSM network with LTS
- Strava data matching
- CSNA ratings at edge level (need to do matching)

Then extend network to include connection to high flow cycle to work ODs


Can we approximate CSNA ratings using observed flow compared to centrality?

In [1]:
import OSM2AT
import osmnx as ox
import geopandas as gpd
import pandas as pd
from geopandas import GeoDataFrame
import matplotlib.pyplot as plt

def min_max_normalize(column):
    min_val = column.min()
    max_val = column.max()
    normalized_column = (column - min_val) / (max_val - min_val)
    return normalized_column

/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/OSM2AT/get_at_networks.py:31: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(use_cache=True, log_console=True, useful_tags_way=utw)


### Part 1 - get Greenwich from OSM2AT

In [7]:
G, edges = OSM2AT.get_cycle_network(None,'mode-rule','ottawa','place','Greenwich, London')

edge_attributes_gdf = GeoDataFrame(edges, crs="EPSG:4326", geometry=edges['geometry'])
edge_attributes_gdf.columns = edge_attributes_gdf.columns.astype(str)
edge_attributes_gdf.to_file("../examples/greenwich/edges.json", driver="GeoJSON")

ox.save_graph_xml(G,'../examples/greenwich/network.xml')
# upload these as github release as follows from system shell:
# gh release upload v1 network_greenwhich.xml
# gh release upload v1 edges.json
# The data is available at: https://github.com/Robinlovelace/netgenhack/releases



/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/OSM2AT/get_at_networks.py:55: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  edge_attributes['cent_x'] = edge_attributes['geometry'].centroid.x
/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/OSM2AT/get_at_networks.py:56: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  edge_attributes['cent_y'] = edge_attributes['geometry'].centroid.y


Imputing Max Speed
Imputing data using method - Mode Rule
Data imputed
Imputing Number of Lanes
Imputing data using method - Mode Rule
Data imputed
Imputing Surface
Index(['osmid', 'oneway', 'highway', 'surface', 'lit', 'bicycle', 'segregated',
       'reversed', 'length', 'name', 'maxspeed', 'motor_vehicle', 'lanes',
       'ref', 'cycleway:left', 'cycleway:both', 'sidewalk',
       'cycleway:left:oneway', 'cycleway', 'cycleway:right', 'junction',
       'cycleway:lane', 'cycleway:surface', 'bridge', 'service', 'access',
       'bus', 'cycleway:right:oneway', 'width', 'tunnel', 'area', 'footway',
       'bicycle:conditional', 'busway', 'est_width', 'path', 'geometry',
       'cent_x', 'cent_y', 'edge_index'],
      dtype='object')
Imputing data using method - mode rule
Data imputed
Calculating LTS using Ottawa Advocacy Group method


/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/osmnx/io.py:361: UserWarning: For the `save_graph_xml` function to behave properly, the graph must have been created with `ox.settings.all_oneway=True`.
  osm_xml._save_graph_xml(
/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/osmnx/osm_xml.py:279: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['False' 'False' 'False' ... 'True' 'True' 'True']' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  gdf_edges.loc[:, "oneway"] = gdf_edges["oneway"].astype(str)


### Part 2 - snap strava data

In [8]:
#Read in edges data
edge_geoms_s = gpd.read_file('../local_data/all_edges_yearly_2019_ride_london/edges_london.shp')
edges_counts_s = pd.read_csv('../local_data/all_edges_yearly_2019_ride_london/edges_london.csv')

In [9]:
id2lts = {}
id2speed = {}
id2roadtype = {}
ids = list(edges['osmid'])

for id in ids:
    id2lts[id] = edges[edges['osmid']==id]['LTS'].mode().values[0]
    id2speed[id] = edges[edges['osmid']==id]['maxspeed'].mode().values[0]
    id2roadtype[id] = edges[edges['osmid']==id]['highway'].mode().values[0]

edges_counts_s['LTS'] = edges_counts_s['osm_reference_id'].map(id2lts)
edges_counts_s['Speed'] = edges_counts_s['osm_reference_id'].map(id2speed)
edges_counts_s['Road Type'] = edges_counts_s['osm_reference_id'].map(id2roadtype)

In [10]:
edges_counts_s_greenwich = edges_counts_s[edges_counts_s['LTS'].notna()]

In [13]:
edges_counts_s_greenwich['forward_people_count_norm'] = min_max_normalize(edges_counts_s_greenwich['forward_people_count'])
for col in ['forward_female_people_count','forward_male_people_count','forward_18_34_people_count','forward_35_54_people_count','forward_55_64_people_count','forward_65_plus_people_count']:
    edges_counts_s_greenwich['att_{}'.format(col)] = min_max_normalize(edges_counts_s_greenwich[col]) / edges_counts_s_greenwich['forward_people_count_norm']

/tmp/ipykernel_1473/3076649626.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edges_counts_s_greenwich['forward_people_count_norm'] = min_max_normalize(edges_counts_s_greenwich['forward_people_count'])
/tmp/ipykernel_1473/3076649626.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edges_counts_s_greenwich['att_{}'.format(col)] = min_max_normalize(edges_counts_s_greenwich[col]) / edges_counts_s_greenwich['forward_people_count_norm']
/tmp/ipykernel_1473/3076649626.py:3: SettingWithCopyWarning: 
A valu

In [18]:
#Output to visualise in QGIS

flow_forward = {}
female_dict = {}
male_dict = {}
age_18_34 = {}
age_35_54 = {}
age_55_64 = {}

for id in ids:
    try:
        female_dict[id] = edges_counts_s_greenwich[edges_counts_s_greenwich['osm_reference_id'] == id]['att_forward_female_people_count'].mode().values[0]
    except:
        female_dict[id] = None
        
    try:
        male_dict[id] = edges_counts_s_greenwich[edges_counts_s_greenwich['osm_reference_id'] == id]['att_forward_male_people_count'].mode().values[0]
    except:
        male_dict[id] = None
        
    try:
        age_18_34[id] = edges_counts_s_greenwich[edges_counts_s_greenwich['osm_reference_id'] == id]['att_forward_18_34_people_count'].mode().values[0]
    except:
        age_18_34[id] = None

    try:
        age_35_54[id] = edges_counts_s_greenwich[edges_counts_s_greenwich['osm_reference_id'] == id]['att_forward_35_54_people_count'].mode().values[0]
    except:
        age_35_54[id] = None
        
    try:
        age_55_64[id] = edges_counts_s_greenwich[edges_counts_s_greenwich['osm_reference_id'] == id]['att_forward_55_64_people_count'].mode().values[0]
    except:
        age_55_64[id] = None
        
        
    try:
        flow_forward[id] = edges_counts_s_greenwich[edges_counts_s_greenwich['osm_reference_id'] == id]['forward_people_count_norm'].mode().values[0]
    except:
        flow_forward[id] = None

edges['flow forward'] = edges['osmid'].map(flow_forward)        
edges['female preference'] = edges['osmid'].map(female_dict)
edges['male preference'] = edges['osmid'].map(male_dict)
edges['age 18 34 preference'] = edges['osmid'].map(age_18_34)
edges['age 35 54 preference'] = edges['osmid'].map(age_35_54)
edges['age 55 64 preference'] = edges['osmid'].map(age_55_64)

### Part 3 - snap CSNA data

In [23]:
csna = gpd.read_file('../local_data/CSNA/tracks_joined.geojson')
csna['osm_id'] = csna['osm_id'].astype(int)
edges = edges.merge(csna[['osm_id','CSNA_level']], left_on = 'osmid', right_on = 'osm_id', how = 'left')

In [29]:
# Output to file
edge_attributes_gdf = GeoDataFrame(edges, crs="EPSG:4326", geometry=edges['geometry'])
edge_attributes_gdf.to_file("../local_data/edges_augmented.json", driver="GeoJSON")

### Part 4 - extend network to popular OD and repeate above + measure centrality